In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [2]:
INDEX = "MS50"
folder_path = Path(f"data{INDEX}")
files = [f.name for f in folder_path.iterdir() if f.is_file()]

In [10]:
posts_per_day = defaultdict(int)   #dict, inizializza a 0 key non esistenti

for file in folder_path.iterdir():
    ticker = file.stem.split("_")[0]
    try:
        df = pd.read_csv(file)
        if not df.empty:
            if "comments" in file.stem:
                df["date"] = pd.to_datetime(df["commentDate"], errors="coerce")
                time_delta = ( df["date"].max() - df["date"].min() ).days + 1
            else:
                df["date"] = pd.to_datetime(df["videoDate"], errors="coerce")
                time_delta = ( df["date"].max() - df["date"].min() ).days + 1 
            
            posts_per_day[f"{ticker}Tot"] += len(df)
            posts_per_day[ticker] += round( len(df) / time_delta , 1 )


    except Exception as e:
        print(f"Errore nel file {file.name}: {e}")

In [11]:
posts_per_day

defaultdict(int,
            {'AALTot': 19,
             'AAL': 0.30000000000000004,
             'ACBTot': 436,
             'ACB': 1.2,
             'AGTot': 52,
             'AG': 2.0,
             'CLOVTot': 312,
             'CLOV': 0.8999999999999999,
             'CRISTot': 5,
             'CRIS': 0.0,
             'CTRMTot': 7,
             'CTRM': 1.0,
             'INOTot': 5,
             'INO': 0.0,
             'NVAXTot': 368,
             'NVAX': 1.0,
             'OPENTot': 203,
             'OPEN': 23.2,
             'RKTTot': 72,
             'RKT': 0.4,
             'SOXLTot': 549,
             'SOXL': 2.0})

In [15]:
data = []
for ticker, count in posts_per_day.items():
    if "Tot" in ticker:
        tot = count
    else:
        data.append({ "Ticker": ticker, "TotElems": tot, "NumElemsPerDay": round( count, 1), "NumElemsPerWeek": round( count*7, 1) })


In [16]:
df = pd.DataFrame( data )
df.to_csv( f"num_elem_{INDEX}.csv", index=None )